In [25]:
import os
from dotenv import load_dotenv

load_dotenv()
DATABASE_URL = os.getenv("PROD_DB_URL")
FILEPATH = "./data/prod.xlsx"


In [26]:
import pandas as pd
from datetime import datetime

def fix_constraints(dataFilepath, sheetName):
    print("fixing constraints for ", sheetName)
    df = pd.read_excel(dataFilepath, sheet_name=sheetName, engine='openpyxl')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    def fix_date_constraints(df):
        curDate = pd.Timestamp(datetime.now().date())
        one_month = pd.Timedelta(days=31)
        one_year = pd.Timedelta(days=365)

        df['start_date'] = pd.to_datetime(df['start_date'])
        df['end_date'] = pd.to_datetime(df['end_date'])

        # find violations
        invalid_start = df['start_date'] <= curDate
        invalid_min_term = df['end_date'] < df['start_date'] + one_month
        invalid_max_term = df['end_date'] > df['start_date'] + one_year

        # on violation, set to some default value
        for idx in df[invalid_start].index:
            print(f"[start_date violation] Row {idx}: start_date={df.at[idx, 'start_date']} <= {curDate} → replaced with {curDate + pd.Timedelta(days=5)}")
            print(f"    end_date also set to {curDate + pd.Timedelta(days=5) + one_month + one_month}")
        df.loc[invalid_start, 'start_date'] = curDate + pd.Timedelta(days=5)
        df.loc[invalid_start, 'end_date'] = df.loc[invalid_start, 'start_date'] + one_month + one_month

        for idx in df[invalid_min_term].index:
            print(f"[min_term violation] Row {idx}: end_date={df.at[idx, 'end_date']} < start_date+1mo={df.at[idx, 'start_date'] + one_month} → replaced with {df.at[idx, 'start_date'] + one_month + one_month}")
        df.loc[invalid_min_term, 'end_date'] = df.loc[invalid_min_term, 'start_date'] + one_month + one_month

        for idx in df[invalid_max_term].index:
            print(f"[max_term violation] Row {idx}: end_date={df.at[idx, 'end_date']} > start_date+1yr={df.at[idx, 'start_date'] + one_year} → replaced with {df.at[idx, 'start_date'] + one_month + one_month}")
        df.loc[invalid_max_term, 'end_date'] = df.loc[invalid_max_term, 'start_date'] + one_month + one_month

        return df

    def fix_min_age_constraint(df):
        if 'age' in df.columns:
            for idx in df[df['age'] < 18].index:
                print(f"[age violation] Row {idx}: age={df.at[idx, 'age']} < 18 → replaced with 20")
            df.loc[df['age'] < 18, 'age'] = 20
        return df

    if sheetName == "Listings":
        df = fix_date_constraints(df)
    elif sheetName == "Renter_Profiles":
        df = fix_date_constraints(df)
        df = fix_min_age_constraint(df)

    # save fixed values back into excel spreadsheet
    with pd.ExcelWriter(dataFilepath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheetName, index=False)

for sheet in ["Listings", "Renter_Profiles"]:
    fix_constraints(FILEPATH, sheet)

fixing constraints for  Listings
fixing constraints for  Renter_Profiles


In [27]:
import pandas as pd
import psycopg2 as pg2
from psycopg2.extras import execute_values

def bulkInsert(sheetName, dataFilepath, reset=False):
    df = pd.read_excel(dataFilepath, sheet_name=sheetName, engine='openpyxl')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    cols = list(df.columns)
    values = list(df.itertuples(index=False, name=None))
    columns_sql = ", ".join(cols)
    placeholders = "(" + ", ".join(["%s"] * len(cols)) + ")"

    query = f"""
        INSERT INTO {sheetName} ({columns_sql})
        VALUES %s
    """

    bigint_columns = ['id', 'user_id', 'locations_id', 'building_type_id']
    for col in bigint_columns:
        if col in df.columns:
            # Check for non-numeric values
            non_numeric = pd.to_numeric(df[col], errors='coerce').isna()
            if non_numeric.any():
                print(f"Non-numeric values in {col}: {df[non_numeric][col].tolist()}")
            
            # Check for out-of-range values
            numeric_vals = pd.to_numeric(df[col], errors='coerce')
            out_of_range = (numeric_vals > 9223372036854775807) | (numeric_vals < -9223372036854775808)
            if out_of_range.any():
                print(f"Out-of-range values in {col}: {df[out_of_range][col].tolist()}")


    with pg2.connect(DATABASE_URL) as conn:
        with conn.cursor() as cur:
            if reset:
                cur.execute(f"TRUNCATE TABLE {sheetName} RESTART IDENTITY CASCADE")

            try:
                execute_values(cur, query, values)
            except Exception as e:
                print(f"insert error: {e}")

    conn.commit()



In [28]:
excel = pd.ExcelFile(FILEPATH)
sheetNames = excel.sheet_names

for sheet in sheetNames:
    print("inserting table: ", sheet)
    bulkInsert(sheet, FILEPATH, True)

inserting table:  Users
inserting table:  Locations
inserting table:  Building_Types
inserting table:  Amenities
inserting table:  Listings
inserting table:  Photos
inserting table:  Renter_Profiles
insert error: bigint out of range

inserting table:  Renter_on_Listing
insert error: insert or update on table "renter_on_listing" violates foreign key constraint "renter_on_listing_renter_profile_id_fkey"
DETAIL:  Key (renter_profile_id)=(850) is not present in table "renter_profiles".

inserting table:  Listing_on_Renter
insert error: insert or update on table "listing_on_renter" violates foreign key constraint "listing_on_renter_renter_profile_id_fkey"
DETAIL:  Key (renter_profile_id)=(925) is not present in table "renter_profiles".

inserting table:  Listing_Amenities
insert error: duplicate key value violates unique constraint "listing_amenities_pkey"
DETAIL:  Key (listing_id, amenity_id)=(577, 17) already exists.



In [ ]:
import psycopg2 as pg2
import pandas as pd
from datetime import datetime

def filterDateConstraints(df):
    curDate = pd.Timestamp(datetime.now().date())
    one_month = pd.Timedelta(days=31)
    one_year = pd.Timedelta(days=365)

    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])

    invalid_start = df['start_date'] <= curDate
    invalid_min_term = df['end_date'] < df['start_date'] + one_month
    invalid_max_term = df['end_date'] > df['start_date'] + one_year

    # on violation, set to some default value
    df.loc[invalid_start, 'start_date'] = curDate + pd.Timedelta(days=1)
    df.loc[invalid_start, 'end_date'] = df.loc[invalid_start, 'start_date'] + one_month

    df.loc[invalid_min_term, 'end_date'] = df.loc[invalid_min_term, 'start_date'] + one_month
    df.loc[invalid_max_term, 'end_date'] = df.loc[invalid_max_term, 'start_date'] + one_year

    return df

def filteTenantAgeConstraint(df):
    invalid_age = df['age'] < 18

    # on violation, set to some default value
    df.loc[invalid_age, 'age'] = 20
    return df

import pandas as pd
import psycopg2 as pg2
from psycopg2.extras import execute_values

def bulkInsert(sheetName, dataFilepath, reset=False):
    df = pd.read_excel(dataFilepath, sheet_name=sheetName, engine='openpyxl')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    

    cols = list(df.columns)
    values = list(df.itertuples(index=False, name=None))
    columns_sql = ", ".join(cols)
    placeholders = "(" + ", ".join(["%s"] * len(cols)) + ")"

    query = f"""
        INSERT INTO {sheetName} ({columns_sql})
        VALUES %s
        ON CONFLICT DO NOTHING;
    """


    with pg2.connect(DATABASE_URL) as conn:
        with conn.cursor() as cur:
            if reset:
                cur.execute(f"TRUNCATE TABLE {sheetName} RESTART IDENTITY CASCADE")

            try:
                execute_values(cur, query, values)
            except Exception as e:
                print(f"insert error: {e}")

    conn.commit()

